**Imports**

In [ ]:
import numpy as np
import pandas as pd

**Reading data**

In [17]:
def parse_data(file_path): # Function for parsing .txt with non-numerical string
    data_sets = {}  # Dictionary to hold the data sets
    current_set = None

    errors = 0

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith("Data set"):  # New data set
                current_set = line  # Update the current data set
                data_sets[current_set] = []  # Initialize an empty list for this set
            elif line:  # Non-empty line, add data to current data set
                try:
                    x, y = map(float, line.split('\t'))
                    data_sets[current_set].append((x, y))
                except Exception as e: # An error usually appears due to the initial string line
                    errors += 1

    return data_sets , errors

data_sets , errors = parse_data('FranksNumbers.txt')

In [20]:
data_sets['Data set 1']

[(10.0, 8.04),
 (8.0, 6.95),
 (13.0, 7.58),
 (9.0, 8.81),
 (11.0, 8.33),
 (14.0, 9.96),
 (6.0, 7.24),
 (4.0, 4.26),
 (12.0, 10.84),
 (7.0, 4.82),
 (5.0, 5.68)]